In [1]:
#PREPARANDO O AMBIENTE
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display
import psycopg2

In [2]:
#CONEXAO COM O BANCO DE DADOS
dbname = '02_etl_python'
user = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'

conn = psycopg2.connect(dbname = dbname,
                        user = user,
                        password = password,
                        host = host,
                        port = port)

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}/{dbname}")
query = 'SELECT * FROM vendas'
df = pd.read_sql(query, engine)
df.head()


,id,data_venda,produto,categoria,valor
0,1,2024-03-01,Notebook Dell,Eletrônicos,4500.0
1,2,2024-03-05,Smartphone Samsung,Eletrônicos,2500.0
2,3,2024-03-10,Camiseta Nike,Moda,120.0
3,4,2024-03-15,Tênis Adidas,Moda,350.0
4,5,2024-03-20,Monitor LG,Eletrônicos,1200.0


In [3]:
#Filtrando as vendas do ultimo mês :
#Primeiro vamos converter a coluna de data para datetime:
df['data_venda'] = pd.to_datetime(df['data_venda'])

#OBTENDO A DATA DE HOJE:
today = pd.to_datetime('today')

#OBTENDO O PRIMEIRO DIA DO ULTIMO MES:
first_day_last_mounth = (today.replace(day=1) - pd.DateOffset(month=1))


#OBTENDO O ULTIMO DIA DO ULTIMO MES:
last_day_mounth = (today.replace(day=1) - pd.Timedelta(days=1))


#FILTRANDO APENAS AS VENDAS DENTRO DESSE INTERVALO:
df_last_mounth = df[(df['data_venda'] >= first_day_last_mounth) & (df['data_venda'] <= last_day_mounth)]
display(df_last_mounth)

,id,data_venda,produto,categoria,valor


In [4]:
#CALCULE O TOTAL DE VENDAS POR CATEGORIA:
sales_per_category = df.groupby('categoria')['valor'].sum().reset_index()
display(sales_per_category)

,categoria,valor
0,Eletrodomésticos,1250.0
1,Eletrônicos,8200.0
2,Moda,470.0
3,Móveis,6850.0


In [5]:
#REMOVA OS REGISTROS COM VALORES NULOS 
nulos_coluna = df.isna().sum()
display(nulos_coluna)


id            0
data_venda    0
produto       0
categoria     0
valor         0
dtype: int64

In [6]:
#CARREGANDO DADOS TRANSFORMADOS DE VOLTA AO POSTGRE
sales_per_category.to_sql("vendas_processadas", engine, if_exists='replace', index=False)
print("Dados carregados com sucesso!")


Dados carregados com sucesso!


In [7]:
#FECHANDO A CONEXAO
conn.close()